In [10]:
import pandas as pd
import numpy as np
gdp_df = pd.read_csv('gdp_per_capita.csv')
gdp_df.rename(columns={'Country (or dependent territory)':'Country'}, inplace=True)
print(gdp_df.head())

       Country   1990   1991   1992   1993   1994   1995   1996   1997   1998  \
0  Afghanistan    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1      Albania    676    408    260    453    736    904  1,010    707    817   
2      Algeria  2,684  1,975  2,033  2,057  1,675  1,627  1,783  1,800  1,772   
3      Andorra    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4       Angola  1,397  1,310    978    688    485    586    731    830    681   

   ...    2020    2021    2022    2023    2024    2025    2026    2027  \
0  ...     517     356     352     411     NaN                           
1  ...   5,353   6,396   6,870   8,299   9,598  10,386  11,211  11,938   
2  ...   3,758   4,170   4,982   5,222   5,579   5,593   5,646   5,681   
3  ...  36,974  41,807  41,379  43,810  45,279  45,995  46,819  47,330   
4  ...   1,990   2,445   3,975   2,967   2,961   2,991   3,030   3,141   

     2028    2029  
0                  
1  12,714  13,543  
2   5,70

filling the - values with 0 in olympic dataset

In [4]:
olympics_df =pd.read_csv('Olympic_medals.csv')
def fixMedals(col):
    return olympics_df[col].replace('-', '0').astype(int)

olympics_df['Gold'] = fixMedals('Gold')
olympics_df['Silver'] = fixMedals('Silver')
olympics_df['Bronze'] = fixMedals('Bronze')
olympics_df['Total'] = fixMedals('Total')


olympics_df.drop(columns="Image URL",inplace=True)

print(olympics_df.head())

        Venue    Country  Gold  Silver  Bronze  Total
0  paris-2024        AIN     1       3       1      5
1  paris-2024    Albania     0       0       2      2
2  paris-2024    Algeria     2       0       1      3
3  paris-2024  Argentina     1       1       1      3
4  paris-2024    Armenia     0       3       1      4


filling nan with after calculating average gdp growth

In [11]:

def fill_nan_with_growth(df):
 
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(str)

    
    df.iloc[:, 1:] = df.iloc[:, 1:].replace(r'^\s*$', np.nan, regex=True)

   
    df.iloc[:, 1:] = df.iloc[:, 1:].replace(',', '', regex=True).astype(float)

    for index, row in df.iterrows():
       
        gdp_values = row[1:]

        valid_gdp = gdp_values.dropna()
        growth_rates = valid_gdp.pct_change().dropna()
        avg_growth_rate = growth_rates.mean()

       
        for i in range(len(gdp_values)):
            if pd.isna(gdp_values[i]):
                if i > 0 and not pd.isna(gdp_values[i - 1]):
                    gdp_values[i] = gdp_values[i - 1] * (1 + avg_growth_rate)
                else:
                  
                    future_index = i + 1
                    while future_index < len(gdp_values) and pd.isna(gdp_values[future_index]):
                        future_index += 1

                    if future_index < len(gdp_values):  
                        future_value = gdp_values[future_index]
                        steps = future_index - i
                        gdp_values[i] = future_value / ((1 + avg_growth_rate) ** steps)  
                    else:
                        gdp_values[i] = np.nanmean(valid_gdp) 
                    
                gdp_values[i] = gdp_values[i].round(1)

    
        df.iloc[index, 1:] = gdp_values

fill_nan_with_growth(gdp_df)

print(gdp_df)

C:\Users\mnuru\AppData\Local\Temp\ipykernel_150352\63015742.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  growth_rates = valid_gdp.pct_change().dropna()
C:\Users\mnuru\AppData\Local\Temp\ipykernel_150352\63015742.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(gdp_values[i]):
C:\Users\mnuru\AppData\Local\Temp\ipykernel_150352\63015742.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc

         Country     1990     1991     1992     1993     1994     1995  \
0    Afghanistan    128.2    133.5    139.0    144.7    150.7    156.9   
1        Albania    676.0    408.0    260.0    453.0    736.0    904.0   
2        Algeria   2684.0   1975.0   2033.0   2057.0   1675.0   1627.0   
3        Andorra  47517.9  47592.4  47667.0  47741.7  47816.5  47891.5   
4         Angola   1397.0   1310.0    978.0    688.0    485.0    586.0   
..           ...      ...      ...      ...      ...      ...      ...   
191      Vietnam    122.0    141.0    179.0    235.0    286.0    359.0   
192    Palestine   1062.0   1101.0   1141.5   1183.5   1227.0   1322.0   
193        Yemen    964.0   1076.0   1268.0   1478.0   1835.0    808.0   
194       Zambia    542.0    475.0    452.0    432.0    432.0    437.0   
195     Zimbabwe   1037.0    936.0    749.0    702.0    714.0    717.0   

        1996     1997     1998  ...     2020     2021     2022     2023  \
0      163.4    170.1    177.1  ... 

In [8]:
list1=[i for i in olympics_df["Country"].unique() if i not in gdp_df["Country"].unique()]
print(list1)

['AIN', 'Cabo Verde', 'Chinese Taipei', 'Cuba', 'Czechia', "Côte d'Ivoire", "Democratic People's Republic of Korea", 'EOR', 'Great Britain', 'Hong Kong, China', 'Islamic Republic of Iran', "People's Republic of China", 'Republic of Korea', 'Republic of Moldova', 'Türkiye', 'United States of America', 'Bermuda', 'ROC', 'Syrian Arab Republic', 'Russian Federation', 'Serbia and Montenegro']


So countries that are in the olympics_df but not in the gdp_df are or are in different name

In [5]:
country_mapping = {
    'Cabo Verde': 'Cape Verde',
    'Chinese Taipei': 'Taiwan',
    'Czechia': 'Czech Republic',
    "Côte d'Ivoire": "Ivory Coast",
    'Hong Kong, China': 'Hong Kong',
    "Islamic Republic of Iran": "Iran",
    "People's Republic of China": "China",
    'Republic of Korea': 'South Korea',
    'Republic of Moldova': 'Moldova',
    'Türkiye': 'Turkey',
    'United States of America': 'United States',
    'Syrian Arab Republic': 'Syria',
    'ROC': 'Russia',
    'Great Britain': 'United Kingdom',
    'Russian Federation': 'Russia',
    'Serbia and Montenegro': 'Serbia',
}
drop_rows={
    'AIN','Cuba','EOR','Bermuda'
}

In [6]:
olympics_df['Country'] = olympics_df['Country'].replace(country_mapping)
olympics_df = olympics_df[~olympics_df['Country'].isin(drop_rows)]

In [7]:
olympics_df

,Venue,Country,Gold,Silver,Bronze,Total
1,paris-2024,Albania,0,0,2,2
2,paris-2024,Algeria,2,0,1,3
3,paris-2024,Argentina,1,1,1,3
4,paris-2024,Armenia,0,3,1,4
5,paris-2024,Australia,18,19,16,53
...,...,...,...,...,...,...
593,sydney-2000,Ukraine,3,10,10,23
594,sydney-2000,United States,37,24,32,93
595,sydney-2000,Uruguay,0,1,0,1
596,sydney-2000,Uzbekistan,1,1,2,4


In [8]:
olympics_df.to_csv('Olympic_medals(after_dataTransformation).csv', index=False)


In [15]:
def drop_year(df, year):
    return df.drop(columns=year)
gdp_df = drop_year(gdp_df, [str(i) for i in range(1990,2030) if (i<1999 or i>2025)])


In [12]:
df_melted = gdp_df.melt(id_vars=['Country'], var_name='Year', value_name='GDP')
print(df_melted.head())

       Country  Year      GDP
0  Afghanistan  1990    128.2
1      Albania  1990    676.0
2      Algeria  1990   2684.0
3      Andorra  1990  47517.9
4       Angola  1990   1397.0


In [14]:
df_melted.to_csv('gdp_per_capita(after_dataTransformation).csv', index=False)